---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
'''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
def get_list_of_university_towns():
    temp = []
    data = open('university_towns.txt')
    for line in data:
        if '[ed' in line:
            state = line.split('[ed')[0]
        else:
            region = line.split('(')[0].strip()
            temp.append([state,region])
    uni_towns = pd.DataFrame(temp,columns=['State','RegionName'])
    return uni_towns

In [37]:
def get_recession_start():
    dat=pd.read_excel('gdplev.xls')
    dat=dat.dropna(axis=1,how='all')
    dat.columns = ['Annual', 'GDP in billions of current dollars_a','GDP in billions of chained 2009 dollars_a',
                   'Quartly','GDP in billions of current dollars','GDP in billions of chained 2009 dollars']
    dat = dat.loc[7:]
    ind = dat[dat.Quartly=='2000q1'].index[0]
    dat=dat.loc[ind:]
    less=0
    for idx,rw in dat.iterrows():
        if rw['GDP in billions of chained 2009 dollars']>less:
            less = rw['GDP in billions of chained 2009 dollars']
        else:
            if dat.loc[idx+1]['GDP in billions of chained 2009 dollars']<rw['GDP in billions of chained 2009 dollars']:
                start = rw['Quartly']
                break
            else:
                less = rw['GDP in billions of chained 2009 dollars']
                
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    return start

In [24]:
def get_recession_end():
    dat=pd.read_excel('gdplev.xls')
    dat=dat.dropna(axis=1,how='all')
    dat.columns = ['Annual', 'GDP in billions of current dollars_a','GDP in billions of chained 2009 dollars_a',
                   'Quartly','GDP in billions of current dollars','GDP in billions of chained 2009 dollars']
    dat = dat.loc[7:]
    start=get_recession_start()
    ind = dat[dat.Quartly==start].index[0]
    dat=dat.loc[ind:]
    more=dat.iloc[0]['GDP in billions of chained 2009 dollars']
    for idx,rw in dat.iterrows():
        if rw['GDP in billions of chained 2009 dollars']<=more:
            more = rw['GDP in billions of chained 2009 dollars']
        else:
            if dat.loc[idx+1]['GDP in billions of chained 2009 dollars']>rw['GDP in billions of chained 2009 dollars']:
                end = dat.loc[idx+1]['Quartly']
                break
            else:
                more = rw['GDP in billions of chained 2009 dollars']
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    return end

In [25]:
def get_recession_bottom():
    dat=pd.read_excel('gdplev.xls')
    dat=dat.dropna(axis=1,how='all')
    dat.columns = ['Annual', 'GDP in billions of current dollars_a','GDP in billions of chained 2009 dollars_a',
                   'Quartly','GDP in billions of current dollars','GDP in billions of chained 2009 dollars']
    dat = dat.loc[7:]
    start = get_recession_start()
    start_ind = dat[dat.Quartly==start].index[0]
    end = get_recession_end()
    end_ind = dat[dat.Quartly==end].index[0]
    dat = dat.loc[start_ind:end_ind]
    low=min(dat['GDP in billions of chained 2009 dollars'])
    bottom = dat[dat['GDP in billions of chained 2009 dollars'] == low]['Quartly'].values[0]
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    return bottom

In [36]:
def convert_housing_data_to_quarters():
    house = pd.read_csv('City_Zhvi_AllHomes.csv')
    house.State.replace(states, inplace=True)
    house=house.set_index( ['State','RegionName'])
    house = house.loc[:,'2000-01':'2016-08']
    house.columns = pd.to_datetime(house.columns)
    house = house.resample('Q',axis=1).mean()
    house = house.rename(columns=lambda x: str(x.to_period('Q')).lower())
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    return house

In [47]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    university =get_list_of_university_towns()
    university = university.drop_duplicates(keep=False)
    start_q=get_recession_start()
    end_q=get_recession_end()
    bottom_q=get_recession_bottom()
    housing = convert_housing_data_to_quarters()
    idx = housing.columns.get_loc(start_q)
    prev_q = housing.columns[idx-1]
    housing['ratio'] = housing[prev_q]/housing[bottom_q]
    housing = housing.drop_duplicates(keep=False)
    df = pd.merge(housing.reset_index(),university,on=university.columns.tolist(),indicator=True,how='outer')
    uni_housing = df[df['_merge']=='both']
    nonuni_housing = df[df['_merge']=='left_only']
    if uni_housing['ratio'].mean() < nonuni_housing['ratio'].mean():
        better = 'university town'
    else:
        better = 'non-university town'    
    val, p= ttest_ind(nonuni_housing['ratio'],uni_housing['ratio'],nan_policy='omit')
    if p < 0.01:
        different = True
    else:
        different= False
    return (different, p, better)
run_ttest()

(True, 0.0027240637047531249, 'university town')